In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [2]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [3]:
img_folder = r'D:\original'
out_folder = r'D:\ToAWS2'

course_list = ['청도','포항','중문','시흥','거창','속리산','춘천']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
  '시흥':'MGC004',
  '거창':'MGC005',
  '속리산':'MGC006',
  '춘천':'MGC007'
}

folder_type = ['rgb','ndvi','thumb']

In [4]:
course_id = course_ids.get('청도')
course_id

'MGC002'

In [5]:
target_folders = glob.glob(os.path.join(out_folder,'**/*'))
target_folders = [x for x in target_folders if course_id in x]
target_folders

['D:\\ToAWS2\\MGC002\\20230907']

In [6]:
target_folder = target_folders[0]

In [7]:
target_date = os.path.split(target_folder)[-1]
course_id, target_date

('MGC002', '20230907')

In [8]:
dataJson_ = glob.glob(os.path.join(out_folder,course_id,target_date,'**/photo*.json'), recursive=True)[0]
dataJson_ 

'D:\\ToAWS2\\MGC002\\20230907\\photo.json'

In [9]:
with open(dataJson_, "r") as _json:
  dataJson= json.load(_json)

In [10]:
len(dataJson)

9891

In [11]:
dataJson[0]

{'id': '20230907150744_1286447571944_356629456389_156910000',
 'originalFileJPG': 'NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_레이크1H\\2023_0907_청도그레이스_레이크1H_0.5m\\20230907_1507-1513_패취류,섬머패취,켄터키,티,잡초,이종잔디,한국잔디\\DJI_20230907150744_0001_D.JPG',
 'originalFileGrp': ['NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_레이크1H\\2023_0907_청도그레이스_레이크1H_0.5m\\20230907_1507-1513_패취류,섬머패취,켄터키,티,잡초,이종잔디,한국잔디\\DJI_20230907150744_0001_MS_G.TIF',
  'NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_레이크1H\\2023_0907_청도그레이스_레이크1H_0.5m\\20230907_1507-1513_패취류,섬머패취,켄터키,티,잡초,이종잔디,한국잔디\\DJI_20230907150744_0001_MS_NIR.TIF',
  'NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_레이크1H\\2023_0907_청도그레이스_레이크1H_0.5m\\20230907_1507-1513_패취류,섬머패취,켄터키,티,잡초,이종잔디,한국잔디\\DJI_20230907150744_0001_MS_R.TIF',
  'NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_레이크1H\

In [12]:
import boto3
from decimal import Decimal
# Get the service resource.
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
table = dynamodb.Table('dsnia_imagelabelsV2')
# table = dynamodb.Table('dsnia_imagelabels')

In [13]:
with table.batch_writer(overwrite_by_pkeys=['metadata', 'id']) as batch:
  for index_, row_ in enumerate(dataJson):
    newrow = {}
    newrow ['id'] = row_['id']
    newrow ['metadata'] = course_id+target_date+row_['info']['area']+row_['info']['desc']
    newrow ['date'] = target_date
    newrow ['area'] = row_['info']['area']
    newrow ['desc'] = row_['info']['desc']
    newrow ['json'] = json.loads(json.dumps(row_), parse_float=Decimal)
    print(index_, end='\r')
    batch.put_item(
      Item= newrow
    )

In [29]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError



response = table.scan()
data = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

KeyboardInterrupt: 

In [ ]:
len(response)

426